# MODEL DESIGN AND TRAINING PIPELINE

In [1]:
import os

os.chdir("../") # currently in ../research so I'm trying to change the working directory to the whole project
%pwd

'/Users/nanakwamekankam/Desktop/NK/Portfolio Projects/End-to-end-ML-Project-with-MLFlow'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [9]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger
import pandas as pd


class ConfigurationManager:
    def __init__(self):
        self,
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [10]:
import pandas as pd
import os
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self, config):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]

        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=25)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-05 11:17:04,000: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-05 11:17:04,003: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-05 11:17:04,009: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-05 11:17:04,039: INFO: common: created directory at: artifacts]
[2025-02-05 11:17:04,040: INFO: common: created directory at: artifacts/model_trainer]
